In [1]:
import sys
print(sys.executable)

/home/mehrzad/anaconda3/envs/dbpedia/bin/python


In [2]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f'\nDevice: {device}')


Device: cuda:0


#
##  starcoderbase-1b

In [4]:
## needs HF token with accepted terms of use for this repo
# !huggingface-cli logout
# !huggingface-cli login
# !huggingface-cli whoami

In [23]:
import os
os.listdir("/home/mehrzad/.cache/huggingface/hub/models--bigcode--starcoderbase-1b/snapshots/109288100c382424b748a488287e4ef7cd7007b3")

['tokenizer.json',
 'generation_config.json',
 'tokenizer_config.json',
 'model.safetensors',
 'merges.txt',
 'config.json',
 'vocab.json',
 'special_tokens_map.json']

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint = "bigcode/starcoderbase-1b"

tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(checkpoint, use_auth_token=True).to(device)


In [5]:
model

GPTBigCodeForCausalLM(
  (transformer): GPTBigCodeModel(
    (wte): Embedding(49152, 2048)
    (wpe): Embedding(8192, 2048)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPTBigCodeBlock(
        (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (attn): GPTBigCodeAttention(
          (c_attn): Linear(in_features=2048, out_features=2304, bias=True)
          (c_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTBigCodeMLP(
          (c_fc): Linear(in_features=2048, out_features=8192, bias=True)
          (c_proj): Linear(in_features=8192, out_features=2048, bias=True)
          (act): PytorchGELUTanh()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((2048,), eps=1e

In [6]:
## candidate prompt format

prompt = """
# write a SPARQL query for;
# what is the population of Italy?
# use following prefixes:

PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>

"""


inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100)
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



# write a SPARQL query for;
# what is the population of Italy?
# use following prefixes:

PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>


SELECT?country?population
WHERE {
?country dbr:population?population.
?country dbo:countryName "Italy".
}
<|endoftext|>


In [31]:
## candidate prompt format

prompt = """
# write a SPARQL query for:
# What was the population of Italy in year 2000 ?

# use following prefixes:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>
"""


inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100)
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



# write a SPARQL query for:
# What was the population of Italy in year 2000?

# use following prefixes:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07


#
## Data 

In [6]:
import pandas as pd

df = pd.read_csv("data/nspm-fine-tuning/train.csv")

# df = df[['prompt', 'completion']]
df

,Unnamed: 0,prompt,completion
0,3382376,# write a SPARQL query for:\n# How many ICD9 d...,select count(*) as ?x where{dbr:Friedreich's_a...
1,2559914,# write a SPARQL query for:\n# What was the fa...,select ?x where{dbr:Destrehan_High_School dbo:...
2,2990831,# write a SPARQL query for:\n# Does adventist ...,ask where{dbr:Adventist_Girls_High_School dbo:...
3,1637967,# write a SPARQL query for:\n# Did aliens area...,ask where{dbr:Aliens_Area dbo:lastPublicationD...
4,1507546,# write a SPARQL query for:\n# Did bridget jon...,ask where{dbr:Bridget_Jones:_The_Edge_of_Reaso...
...,...,...,...
99995,7260453,# write a SPARQL query for:\n# What is the pop...,select ?x where{dbr:Southern_Yukaghir_language...
99996,6784536,# write a SPARQL query for:\n# Is washington c...,"ask where{dbr:Mitchell_County,_North_Carolina ..."
99997,3733762,# write a SPARQL query for:\n# How many marrie...,select count(*) as ?x where{dbr:Bernardine_Doh...
99998,7637742,# write a SPARQL query for:\n# What are the ba...,select ?x where{dbr:Battle_of_Magdala dbo:resu...


In [7]:
for index, row in df.sample(n=5).iterrows():
    print(f"Prompt:\n{row['prompt']}\n\nSPARQL:\n{row['completion']}\n{'='*80}")

Prompt:
# write a SPARQL query for:
# How many leaderFunction did deutsche sammlung von mikroorganismen und zellkulturen have ?

SPARQL:
select count(*) as ?x where{dbr:Deutsche_Sammlung_von_Mikroorganismen_und_Zellkulturen dbo:leaderFunction ?x }
Prompt:
# write a SPARQL query for:
# Who is tuesday's guest's subsidiary ?

SPARQL:
select ?x where{dbr:Tuesday's_Guest dbo:productionCompany ?x1 . ?x1 dbo:subsidiary ?x }
Prompt:
# write a SPARQL query for:
# How many fuel systems did ferrari v8 f1 engine have ?

SPARQL:
select count(*) as ?x where{dbr:Ferrari_V8_F1_engine dbo:fuelSystem ?x }
Prompt:
# write a SPARQL query for:
# Did superior vesical artery branch come from ileal arteries ?

SPARQL:
ask where{dbr:Ileal_arteries dbo:branchFrom dbr:Superior_vesical_artery }
Prompt:
# write a SPARQL query for:
# What colour used in school gann academy ?

SPARQL:
select ?x where{dbr:Gann_Academy dbo:officialSchoolColour ?x }


In [8]:
## Load Local CSV Filex to Dataset Format using HugginFace datasets library 

from datasets import load_dataset

dataset = load_dataset('csv', data_files={
    'train': '/home/mehrzad/repos/mehrz/dbpedia/data/nspm-fine-tuning/train.csv', 
    'val': '/home/mehrzad/repos/mehrz/dbpedia/data/nspm-fine-tuning/val.csv'
})


print(dataset)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /home/mehrzad/.cache/huggingface/datasets/csv/default-a3e3d434e059fc25/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'prompt', 'completion'],
        num_rows: 100000
    })
    val: Dataset({
        features: ['Unnamed: 0', 'prompt', 'completion'],
        num_rows: 20000
    })
})


In [9]:
print(dataset['train'][0]['prompt'])
print(dataset['train'][0]['completion'])

# write a SPARQL query for:
# How many ICD9 did friedreich's ataxia have ?
select count(*) as ?x where{dbr:Friedreich's_ataxia dbo:icd9 ?x }


In [10]:
print(dataset['val'][0]['prompt'])
print(dataset['val'][0]['completion'])

# write a SPARQL query for:
# Did 1946 vancouver island earthquake lose any of its comrades ?
ask where{dbr:1946_Vancouver_Island_earthquake dbo:casualties ?x }


In [12]:
import argparse
import os

import torch
from accelerate import Accelerator
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, set_peft_model_state_dict
from torch.utils.data import IterableDataset
from tqdm import tqdm
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, logging, set_seed
from transformers import TrainerCallback, TrainingArguments, TrainerState, TrainerControl
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR

In [25]:
class ConstantLengthDataset(IterableDataset):
    """
    Iterable dataset that returns constant length chunks of tokens from stream of text files.
        Args:
            tokenizer (Tokenizer): The processor used for proccessing the data.
            dataset (dataset.Dataset): Dataset with text files.
            infinite (bool): If True the iterator is reset after dataset reaches end else stops.
            seq_length (int): Length of token sequences to return.
            num_of_sequences (int): Number of token sequences to keep in buffer.
            chars_per_token (int): Number of characters per token used to estimate number of tokens in text buffer.
    """

    def __init__(
        self,
        tokenizer,
        dataset,
        infinite=False,
        seq_length=1024,
        num_of_sequences=1024,
        chars_per_token=3.6,
        input_column_name="prompt",
        output_column_name="completion"
    ):
        self.tokenizer = tokenizer
        self.concat_token_id = tokenizer.eos_token_id if tokenizer.eos_token_id is not None else 0
        self.dataset = dataset
        self.seq_length = seq_length
        self.infinite = infinite
        self.current_size = 0
        self.max_buffer_size = seq_length * chars_per_token * num_of_sequences
        self.input_column_name = input_column_name
        self.output_column_name = output_column_name

    def __iter__(self):
        iterator = iter(self.dataset)
        more_examples = True
        while more_examples:
            buffer, buffer_len = [], 0
            while True:
                if buffer_len >= self.max_buffer_size:
                    break
                try:
                    buffer.append(prepare_sample_text(next(iterator), self.input_column_name, self.output_column_name))
                    buffer_len += len(buffer[-1])
                except StopIteration:
                    if self.infinite:
                        iterator = iter(self.dataset)
                    else:
                        more_examples = False
                        break
            tokenized_inputs = self.tokenizer(buffer, truncation=False)["input_ids"]
            all_token_ids = []
            for tokenized_input in tokenized_inputs:
                all_token_ids.extend(tokenized_input + [self.concat_token_id])
            for i in range(0, len(all_token_ids), self.seq_length):
                input_ids = all_token_ids[i : i + self.seq_length]
                if len(input_ids) == self.seq_length:
                    self.current_size += 1
                    yield {
                        "input_ids": torch.LongTensor(input_ids),
                        "labels": torch.LongTensor(input_ids),
                    }



                    
                    
def prepare_sample_text(example, input_column_name="prompt", output_column_name="completion"):
    """Prepare the text from a sample of the dataset."""
    text = f"Question: {example[input_column_name]}\n\nAnswer: {example[output_column_name]}"
    return text                    
                    

    
                    
def chars_token_ratio(dataset, tokenizer, input_column_name="prompt", output_column_name="completion", nb_examples=400):
    """
    Estimate the average number of characters per token in the dataset.
    """
    total_characters, total_tokens = 0, 0
    for _, example in tqdm(zip(range(nb_examples), iter(dataset)), total=nb_examples):
        text = prepare_sample_text(example, input_column_name, output_column_name)
        total_characters += len(text)
        if tokenizer.is_fast:
            total_tokens += len(tokenizer(text).tokens())
        else:
            total_tokens += len(tokenizer.tokenize(text))

    return total_characters / total_tokens                    
 
    
    
                    
def create_datasets(tokenizer):
    dataset = load_dataset(
        'csv', data_files={
            'train': 'data/nspm_100k_train.csv',
            'test': 'data/nspm_100k_test.csv'
}
#         args.dataset_name,
#         data_dir=args.subset,
#         split=args.split,
#         use_auth_token=True,
#         num_proc=None, ##
#         streaming=args.streaming,
    )
#     if args.streaming:
#         print("Loading the dataset in streaming mode")
#         valid_data = dataset.take(args.size_valid_set)
#         train_data = dataset.skip(args.size_valid_set)
#         train_data = train_data.shuffle(buffer_size=args.shuffle_buffer, seed=args.seed)
#     else:
    train_data = dataset["train"]
    valid_data = dataset["test"]
    print(f"Size of the train set: {len(train_data)}. Size of the validation set: {len(valid_data)}")

    chars_per_token = chars_token_ratio(train_data, tokenizer, "prompt", "completion")
    print(f"The character to token ratio of the dataset is: {chars_per_token:.2f}")

    train_dataset = ConstantLengthDataset(
        tokenizer,
        train_data,
        infinite=True,
        seq_length=2048, ##
        chars_per_token=chars_per_token,
        input_column_name="prompt", ##
        output_column_name="completion" ##
    )
    valid_dataset = ConstantLengthDataset(
        tokenizer,
        valid_data,
        infinite=False,
        seq_length=2048, ##
        chars_per_token=chars_per_token,
        input_column_name="prompt", ##
        output_column_name="completion" ##
    )
    return train_dataset, valid_dataset


In [26]:
train_dataset, valid_dataset = create_datasets(tokenizer)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /home/mehrzad/.cache/huggingface/datasets/csv/default-2ecec2e06c3917ec/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Size of the train set: 70000. Size of the validation set: 30000


100%|███████████████████████████████████████| 400/400 [00:00<00:00, 6637.63it/s]

The character to token ratio of the dataset is: 2.98


In [27]:
valid_dataset.dataset[0]

{'prompt': '# write a SPARQL query for:\n# What is the power output of a mercedes-benz m117 engine machine ?\n\n# use following prefixes:\nPREFIX dbr: <http://dbpedia.org/resource/>\nPREFIX dbo: <http://dbpedia.org/ontology/>',
 'completion': 'select ?x where{dbr:Mercedes-Benz_M117_engine dbo:powerOutput ?x }'}

#
## Training for fine-tuning

In [3]:
%%time
!python finetune-starcoder-1b.py\
--model_path="bigcode/starcoderbase-1b"\
--eos_token_id=0

/bin/bash: /home/mehrzad/anaconda3/envs/tf-2.9/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Found cached dataset csv (/home/mehrzad/.cache/huggingface/datasets/csv/default-2ecec2e06c3917ec/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
100%|███████████████████████████████████████████| 2/2 [00:00<00:00, 1116.25it/s]
Size of the train set: 70000. Size of the validation set: 30000
100%|███████████████████████████████████████| 400/400 [00:00<00:00, 6106.67it/s]
The character to token ratio of the dataset is: 2.98
Loading the model
/home/mehrzad/anaconda3/envs/dbpedia/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
trainable params: 7176192 || all params: 1144383488 || trainable%: 0.6270793029827428
Starting main loop
Training...
/home/mehrzad/anaconda3/envs/dbpedia/li

In [39]:
%%time
!python merge_peft_adapters.py\
--base_model_name_or_path="bigcode/starcoderbase-1b"\
--peft_model_path="./checkpoints/checkpoint-2000/"


Model saved to bigcode/starcoderbase-1b-merged


## 2nd round

In [2]:
import sys
print(sys.executable)

/home/mehr-shahin/repos/dbp/neural-qa/gsoc/mehrzad/starcoder-1b/bin/python3


In [3]:
%%time
!python finetune-starcoder-1b.py\
--model_path="bigcode/starcoderbase-1b"\
--seq_length=512\
--batch_size=2\
--eos_token_id=0\
--save_freq=2000\
--seed=42




Using CUDA device: NVIDIA GeForce RTX 3090
Memory Allocated: 0.00 MB
Memory Reserved: 0.00 MB



Found cached dataset csv (/home/mehr-shahin/.cache/huggingface/datasets/csv/default-3615beb83b6f428b/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|███████████████████████████████████████████| 2/2 [00:00<00:00, 1019.77it/s]
Size of the train set: 100000. Size of the validation set: 20000
100%|███████████████████████████████████████| 400/400 [00:00<00:00, 6434.24it/s]
The character to token ratio of the dataset is: 2.89
Loading the model
/home/mehr-shahin/repos/dbp/neural-qa/gsoc/mehrzad/starcoder-1b/lib/python3.11/site-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
trainable params: 7176192 || all params: 1144383488 || trainable%: 0.6270793029827428
Starting main loop
Training...
/home/mehr-shahin/repos/dbp/ne

In [4]:
%%time
!python merge_peft_adapters.py\
--base_model_name_or_path="bigcode/starcoderbase-1b"\
--peft_model_path="./checkpoints/final_checkpoint/"

Model saved to bigcode/starcoderbase-1b-merged
CPU times: user 247 ms, sys: 58.6 ms, total: 306 ms
Wall time: 40.4 s


# 
## Evaluate fine-tuned model with QALD-9-plus


### Load fine-tuned checkpoints


In [16]:
import os

os.listdir("./checkpoints/checkpoint-2000/")

['optimizer.pt',
 'scheduler.pt',
 'pytorch_model.bin',
 'adapter_model.bin',
 'README.md',
 'rng_state.pth',
 'trainer_state.json',
 'training_args.bin',
 'adapter_config.json']

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint="bigcode/starcoderbase-1b-merged"

tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(checkpoint, use_auth_token=True).to(device)


In [6]:
## candidate prompt format

prompt = """
# write a SPARQL query for:
# What is the foundation of diocese of birobidzhan ?

# use following prefixes:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>
"""


inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100)
print(tokenizer.decode(outputs[0]))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



# write a SPARQL query for:
# What is the foundation of diocese of birobidzhan?

# use following prefixes:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>

Answer: select?x where{dbr:Birobidzhan dbo:foundation?x1.?x1 dbo:dbrId?x }<|endoftext|>


In [7]:
## candidate prompt format

prompt = """
# write a SPARQL query for:
# What is the capital of Canada ?

# use following prefixes:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>
"""


inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100)
print(tokenizer.decode(outputs[0]))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



# write a SPARQL query for:
# What is the capital of Canada?

# use following prefixes:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>

Answer: select?x where{dbr:Canada dbo:capital?x1.?x1 dbo:capital?x }<|endoftext|>


In [9]:
## candidate prompt format

prompt = """
# write a SPARQL query for:
# What was the population of Italy in 2000 ?

# use following prefixes:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>
"""


inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100)
print(tokenizer.decode(outputs[0]))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



# write a SPARQL query for:
# What was the population of Italy in 2000?

# use following prefixes:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>

Answer: select?x where{dbr:Italy dbo:population?x1.?x1 dbo:year?x }<|endoftext|>


In [10]:
## candidate prompt format

prompt = """
# write a SPARQL query for:
# What is the population of France ?

# use following prefixes:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>
"""


inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100)
print(tokenizer.decode(outputs[0]))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



# write a SPARQL query for:
# What is the population of France?

# use following prefixes:
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>

Answer: select?x where{dbr:France dbo:population?x }<|endoftext|>


In [ ]:
## todo
prompt.stirp

In [16]:
import json

with open('data/qald_9_plus_train_dbpedia.json', 'r') as f:
    qald_9_plus_data = json.load(f)
    
qald_9_plus_data['questions'][0]

{'id': '1',
 'question': [{'language': 'en', 'string': 'List all boardgames by GMT.'},
  {'language': 'de', 'string': 'Liste die Brettspiele von GMT auf.'},
  {'language': 'de', 'string': 'Zeige mir alle Brettspiele von GMT.'},
  {'language': 'ru', 'string': 'Перечислите все игры GMT.'},
  {'language': 'lt', 'string': 'Išvardinkite visus stalo žaidimus pagal GMT.'},
  {'language': 'uk', 'string': 'Перерахуйте всі ігри GMT.'},
  {'language': 'lt', 'string': 'Išvardykite visus GMT žaidimus.'},
  {'language': 'fr', 'string': 'Listez tous les jeux de société de GMT.'},
  {'language': 'es',
   'string': '¿Qué juegos de mesa fueron hechos por GMT?',
   'keywords': 'juego de mesa ,  GMT '}],
 'query': {'sparql': 'PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> SELECT ?uri WHERE { ?uri dbo:publisher res:GMT_Games }'},
 'answers': [{'head': {'link': [], 'vars': ['uri']},
   'results': {'bindings': [{'uri'

In [17]:
# extracting pairs of question/sparql    
    
nl_questions, sparql_queries =[],[]

for data in qald_9_plus_data['questions']:
    nl_questions.append(
        next((item['string'] for item in data['question'] if item['language'] == 'en'), None)
    )
    sparql_queries.append(data['query']['sparql'])
    

print(f'{len(nl_questions)} pairs extracted from train set: ', "\n")
print(nl_questions[0])
print('\n', sparql_queries[0].replace(">", '>\n'))

408 pairs extracted from train set:  

List all boardgames by GMT.

 PREFIX dbo: <http://dbpedia.org/ontology/>
 PREFIX res: <http://dbpedia.org/resource/>
 PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
 SELECT ?uri WHERE { ?uri dbo:publisher res:GMT_Games }


# 

### Calculate BELUE score

In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_metric
import pandas as pd
import torch

import argparse


def get_args():
    parser = argparse.ArgumentParser()
    parser = argparse.ArgumentParser()
    parser.add_argument("--model_name_or_path", type=str)
    parser.add_argument("--data_path", type=str, default="/")
    parser.add_argument("--metric_name", type=str, default="bleu")
    parser.add_argument("--input_column_name", type=str, default="prompt")
    parser.add_argument("--output_column_name", type=str, default="completion")
    return parser.parse_args()

 
    
def read_dataset(path):
    data = pd.read_csv(path)
    return data



def generate_SPARQL(model, tokenizer, prompt):
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
    input_ids = inputs["input_ids"].to(model.device)
    attention_mask = inputs["attention_mask"].to(model.device)
    
    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=250,
        num_beams=5,
        pad_token_id=tokenizer.eos_token_id,
        early_stopping=True
    )
    
    return tokenizer.decode(outputs[0])



def calculate_bleu(actual, generated, metric_name):
    bleu_metric = load_metric(metric_name)
    bleu_score = bleu_metric.compute(predictions=[generated.split()], references=[[actual.split()]])
    return bleu_score['bleu']



def main():
    
    args = get_args()

    data = read_dataset(args.data_path)

    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    
    tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path)
    tokenizer.pad_token = tokenizer.eos_token   # Set pad token to EOS token
    model = AutoModelForCausalLM.from_pretrained(
        args.model_name_or_path, use_auth_token=True).to(device)

    
    print(f'\n\nTokenizer and model successfully loaded on:\t{device}')
    
    print(f'\nCalculating scores with <{args.metric_name}> as evaluation metric\n...'+"\n"*2)
    
    results = []
    total_bleu_score = 0

    for idx, row in data.iterrows():
        prompt, actual_sparql = row[args.input_column_name], row[args.output_column_name]
        generated_sparql = generate_SPARQL(model, tokenizer, prompt)

        bleu_score = calculate_bleu(actual_sparql, generated_sparql, args.metric_name)

        results.append((prompt, actual_sparql, generated_sparql, bleu_score))

        total_bleu_score += bleu_score

    average_bleu_score = total_bleu_score / len(data)

    
    print("\n"+"*"*50 +f"Results for {args.model_name_or_path}:\n", results)
    print(f"\nAverage BLEU score for {args.model_name_or_path}: ", average_bleu_score)


    # delete model & tokenizer and clear cache 
    del model
    del tokenizer

    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        
        

if __name__ == "__main__" :
    main()            



In [11]:
df_test = pd.read_csv("data/nspm_100k_test.csv")

df_sample = df_test.sample(n=500, random_state=0)
df_sample.to_csv("data/nspm_test_500.csv", index=False)

In [28]:
%%time

!python evaluate_model.py\
--model_name_or_path="bigcode/starcoderbase-1b-merged"\
--data_path="./data/nspm_test_500.csv"\
--metric_name="bleu"\
--verbose

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
INFO:root:Loading tokenizer and model on cuda:0.

INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmpq78wczpo
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmpq78wczpo/_remote_module_non_scriptable.py

Calculating scores with <bleu> as evaluation metric
...


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/home/mehrzad/repos/mehrz/dbpedia/neural-qa-mehrzadshm/gsoc/mehrzad/evaluate_model.py:47: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new 

In [3]:
%%time

!python evaluate_model.py\
--model_name_or_path="bigcode/starcoderbase-1b-merged"\
--data_path="./data/nspm_test_500.csv"\
--metric_name="bleu"\
--report_file_name="eval_results_02"\
--verbose

INFO:root:Loading tokenizer and model on cuda:0.

INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmpa78o8_5c
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmpa78o8_5c/_remote_module_non_scriptable.py

Calculating scores with <bleu> as evaluation metric
...


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/home/mehrzad/repos/mehrz/dbpedia/neural-qa-mehrzadshm/gsoc/mehrzad/evaluate_model.py:52: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric(metric_name)

********************************************************************************
Results written to eval_results.csv

Average BLEU score for 500 test samples:  0.01604976731943413
CPU times: user 8.28 s, sys: 1.46 s, total: 9

In [5]:
%%time

!python evaluate_model.py\
--model_name_or_path="bigcode/starcoderbase-1b"\
--data_path="./data/nspm_test_500.csv"\
--metric_name="bleu"\
--report_file_name="eval_results_starcoder_base_1b"\
--verbose

INFO:root:Loading tokenizer and model on cuda:0.

INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmp__s_mmtm
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmp__s_mmtm/_remote_module_non_scriptable.py

Calculating scores with <bleu> as evaluation metric
...


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/home/mehrzad/repos/mehrz/dbpedia/neural-qa-mehrzadshm/gsoc/mehrzad/evaluate_model.py:52: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric(metric_name)

********************************************************************************
Results written to eval_results.csv

Average BLEU score for 500 test samples:  0.0
CPU times: user 19.3 s, sys: 2.81 s, total: 22.1 s
Wall time:

In [16]:
class SPARQLPostProcessor:
    
    ## TODO

    # deal with spaces


    # for now, remove things based on "Answer:" token
    def remove_from_token(self, generated_output, reference_token="Answer:"):
        return generated_output.replace(reference_token, '').strip()

    
    def post_process(self, query):
        query = self.remove_from_token(query)
        #
        #
        return query
    
  





def generate_SPARQL(model, tokenizer, prompt):
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
    input_ids = inputs["input_ids"].to(model.device)
    attention_mask = inputs["attention_mask"].to(model.device)
    
    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=250,
        num_beams=5,
        pad_token_id=tokenizer.eos_token_id,
        early_stopping=True
    ) 
    
    return tokenizer.decode(outputs[0])



tokenizer.pad_token = tokenizer.eos_token   # Set pad token to EOS token

In [25]:
prompt = """
# write a SPARQL query for;
# what is the population of Italy?
# use following prefixes:

PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>
"""


post_processor = SPARQLPostProcessor()

generated_sparql = generate_SPARQL(model, tokenizer, prompt)
print(generated_sparql)

print("\nAfter")

clean_generated_sparql = post_processor.post_process(generated_sparql[len(prompt):])
print(clean_generated_sparql)



# write a SPARQL query for;
# what is the population of Italy?
# use following prefixes:

PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>

Answer: select?x where{dbr:Italy dbo:population?x }<|endoftext|>

After
select?x where{dbr:Italy dbo:population?x }<|endoftext|>


In [11]:
print(generated_sparql)

'\n# write a SPARQL query for;\n# what is the population of Italy?\n# use following prefixes:\n\nPREFIX dbr: <http://dbpedia.org/resource/>\nPREFIX dbo: <http://dbpedia.org/ontology/>\n\nAnswer: select?x where{dbr:Italy dbo:population?x }<|endoftext|>'

In [22]:
generated_sparql[len(prompt):]

'\nAnswer: select?x where{dbr:Italy dbo:population?x }<|endoftext|>'

In [20]:
prompt

'\n# write a SPARQL query for;\n# what is the population of Italy?\n# use following prefixes:\n\nPREFIX dbr: <http://dbpedia.org/resource/>\nPREFIX dbo: <http://dbpedia.org/ontology/>\n'

# 
## Evaluate 2nd fine-tuned model with QALD-9-plus


### Load fine-tuned checkpoints


In [63]:
%%time

!python evaluate_model.py\
--model_name_or_path="bigcode/starcoderbase-1b-merged"\
--data_path="./data/nspm-fine-tuning/test.csv"\
--metric_name="bleu"\
--report_file_name="eval_results_starcoder_base_1b_02"\
--verbose

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
INFO:root:Loading tokenizer and model on cuda:0.

INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmpqw9bxtp8
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmpqw9bxtp8/_remote_module_non_scriptable.py

Calculating scores with <bleu> as evaluation metric
...


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/home/mehr-shahin/repos/dbp/neural-qa/gsoc/mehrzad/evaluate_model.py:53: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluat

In [69]:
import pandas as pd

df = pd.read_csv("eval_results_starcoder_base_1b_02.csv")
df
                     

,Prompt,Actual SPARQL,Generated SPARQL,BLEU Score (Generated),Post-Processed SPARQL,BLEU Score (Post-Processed)
0,# write a SPARQL query for:\n# What is the Lin...,select ?x where{dbr:Karl_Marx dbo:birthPlace ?...,\nAnswer: select?x where{dbr:Karl_Marx dbo:bir...,0.000000,select ?x where{dbr:Karl_Marx dbo:birthPlace ?...,0.330101
1,# write a SPARQL query for:\n# Is diocesan col...,ask where{dbr:Morgan_Newman dbo:school dbr:Dio...,\nAnswer: ask where{dbr:Morgan_Newman dbo:scho...,0.508133,ask where{dbr:Morgan_Newman dbo:school dbr:Dio...,1.000000
2,# write a SPARQL query for:\n# What is the cou...,select ?x where{dbr:Canário dbo:birthPlace ?x1...,\nAnswer: select?x where{dbr:Canário dbo:birth...,0.000000,select ?x where{dbr:Canário dbo:birthPlace ?x1...,0.534174
3,# write a SPARQL query for:\n# What is the por...,select ?x where{dbr:Stella_Bonasera dbo:portra...,\nAnswer: select?x where{dbr:Stella_Bonasera d...,0.000000,select ?x where{dbr:Stella_Bonasera dbo:portra...,0.534174
4,# write a SPARQL query for:\n# Has south afric...,ask where{dbr:South_African_Class_16E dbo:rebu...,\nAnswer: ask where{dbr:South_African_Class_16...,0.000000,ask where{dbr:South_African_Class_16E dbo:rebu...,1.000000
...,...,...,...,...,...,...
19995,# write a SPARQL query for:\n# How many diamet...,select count(*) as ?x where{dbr:Volterra_(crat...,\nAnswer: select count(*) as?x where{dbr:Volte...,0.000000,select count(*) as ?x where{dbr:Volterra dbo:d...,0.500000
19996,# write a SPARQL query for:\n# How is hyatt re...,select ?x where{dbr:Hyatt_Regency_Chicago dbo:...,\nAnswer: select?x where{dbr:Hotel_Inglaterra ...,0.000000,select ?x where{dbr:Hotel_Inglaterra dbo:floor...,0.000000
19997,# write a SPARQL query for:\n# Who is thomas d...,select ?x where{dbr:Thomas_Dudley dbo:father ?...,\nAnswer: select?x where{dbr:Thomas_Dudley dbo...,0.000000,select ?x where{dbr:Thomas_Dudley dbo:father ?...,0.317622
19998,# write a SPARQL query for:\n# What are most o...,select count(*) as ?x where{dbr:Schuylkill_Ars...,\nAnswer: select?x where{dbr:Schuylkill_Arsena...,0.000000,select ?x where{dbr:Schuylkill_Arsenal_Railroa...,0.569836


In [72]:
print("\n")
print("Descriptive Statistics for BLEU Scores (Generated):")
print(df['BLEU Score (Generated)'].describe())
print("\n")
print("Descriptive Statistics for BLEU Scores (Post-Processed):")
print(df['BLEU Score (Post-Processed)'].describe())



Descriptive Statistics for BLEU Scores (Generated):
count    20000.000000
mean         0.029052
std          0.116592
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          0.759836
Name: BLEU Score (Generated), dtype: float64


Descriptive Statistics for BLEU Scores (Post-Processed):
count    20000.000000
mean         0.437779
std          0.413116
min          0.000000
25%          0.000000
50%          0.386275
75%          1.000000
max          1.000000
Name: BLEU Score (Post-Processed), dtype: float64


In [92]:
# Filter rows where 'BLEU Score (Post-Processed)' is 0
zero_score_df = df[df['BLEU Score (Post-Processed)'] == 0]

# error analysis based on sample rows where BLEU Score (Post-Processed) is 0
sampled_df = zero_score_df.sample(n=min(100, len(zero_score_df)))

for index, row in sampled_df[['Prompt','Actual SPARQL', 'Post-Processed SPARQL']].iterrows():
    print("Prompt:    ", row["Prompt"][28:], "\n")
    print("Actual:    ", row["Actual SPARQL"])
    print("Processed: ", row["Post-Processed SPARQL"])
    print("-"*100)  

Prompt:     # Is the emerald number of parking spaces of claridge icon ? 

Actual:     ask where{dbr:Claridge_Icon dbo:numberOfParkingSpaces dbr:The_Emerald_(building) }
Processed:  ask where{dbr:Claridge_Icon dbo:numberOfParkingSpaces dbr:Emerald }
----------------------------------------------------------------------------------------------------
Prompt:     # Did the george houston house have the NRHP reference number ? 

Actual:     ask where{dbr:George_Houston_House dbo:nrhpReferenceNumber ?x }
Processed:  ask where{dbr:The_George_Houston_House dbo:nrhpReferenceNumber ?x }
----------------------------------------------------------------------------------------------------
Prompt:     # How should I know the ingredient names for mache ? 

Actual:     select ?x where{dbr:Mache_(food) dbo:ingredientName ?x }
Processed:  select ?x where{dbr:Mache dbo:ingredientName ?x }
----------------------------------------------------------------------------------------------------
Prompt:     # W

In [90]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f'\nDevice: {device}')


Device: cuda:0


In [91]:
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint="bigcode/starcoderbase-1b-merged"

tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(checkpoint, use_auth_token=True).to(device)


In [23]:
## candidate prompt format

prompt = "Question: # write a SPARQL query for:\n# What are cadolzburg area code ?"


inputs = tokenizer.encode_plus(prompt.strip(), return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100)
print(tokenizer.decode(outputs[0]))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question: # write a SPARQL query for:
# What are cadolzburg area code?

Answer: select?x where{dbr:Cadolzburg dbo:areaCode?x }<|endoftext|>


In [24]:
## candidate prompt format

prompt = "Question: # write a SPARQL query for:\n# How many province did avola grow in ?"


inputs = tokenizer.encode_plus(prompt.strip(), return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100)
print(tokenizer.decode(outputs[0]))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question: # write a SPARQL query for:
# How many province did avola grow in?

Answer: select count(*) as?x where{dbr:Avola dbo:province?x }<|endoftext|>


In [16]:
import pandas as pd
df_test = pd.read_csv("data/nspm-fine-tuning/test.csv")

df_sample = df_test.sample(n=10, random_state=0)

for i in df_sample['prompt'].to_list():
    print(i, "\n")

# write a SPARQL query for:
# What's the percentage of area water in bert cremean's Death place ? 

# write a SPARQL query for:
# Did calabritto have saint ? 

# write a SPARQL query for:
# How many ideologies did ganatantri dal have ? 

# write a SPARQL query for:
# Who is the second driver of 1976 race of champions ? 

# write a SPARQL query for:
# What is the ship beam of voyager of the seas as well as uss fort worth ? 

# write a SPARQL query for:
# How many province did avola grow in ? 

# write a SPARQL query for:
# What are cadolzburg area code ? 

# write a SPARQL query for:
# What ISO code for catalan language ? 

# write a SPARQL query for:
# What is the population of the city of sohawa tehsil ? 

# write a SPARQL query for:
# What is the revenue of zale corporation ? 



In [ ]:
select?x where{dbr:Bert_Cremean dbo:deathPlace?x1.?x1 dbo:percentageOfAreaWater?x }<|endoftext|>

In [21]:
for i in df_sample['completion'].to_list():
    print(i, "\n")

select ?x where{dbr:Bert_Cremean dbo:deathPlace ?x1 . ?x1 dbo:percentageOfAreaWater ?x } 

ask where{dbr:Calabritto dbo:saint ?x } 

select count(*) as ?x where{dbr:Ganatantri_Dal dbo:ideology ?x } 

select ?x where{dbr:1976_Race_of_Champions dbo:secondDriver ?x } 

select ?x where{dbr:Voyager_of_the_Seas dbo:shipBeam ?x . dbr:USS_Fort_Worth dbo:shipBeam ?x } 

select count(*) as ?x where{dbr:Avola dbo:province ?x } 

select count(*) as ?x where{dbr:Cadolzburg dbo:areaCode ?x } 

select count(*) as ?x where{dbr:Catalan_language dbo:iso6391Code ?x } 

select ?x where{dbr:Sohawa_Tehsil dbo:populationRural ?x } 

select count(*) as ?x where{dbr:Zale_Corporation dbo:revenue ?x } 

